In [4]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
import base64
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from CRUD_Trembley import CRUD_shelter as cs





###########################
# Data Manipulation / Model
###########################
username = "aacuser2"
password = "meeko321"
shelter = cs(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))),
    html.Center(html.P("by Matthew Trembley")),
    html.Hr(),
    html.Div(
        dcc.RadioItems( #Radio buttons for filtering
            id = "filter",
            options = [
                {"label" : "Water Rescue", "value" : "wr"},
                {"label" : "Mountain or Wilderness Rescue", "value" : "mtn"},
                {"label" : "Disaster or Individual Tracking", "value" : "dis"},
                {"label" : "Reset", "value" : "RESET"}],
                value = "RESET",
            labelStyle = {"display" : "inline-block"}
        )
        

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [0],
        page_action = "native",
        page_current = 0,
        page_size = 10,
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
        style={'display' : 'flex'},
            children=[
                dcc.Graph(
                id='graph-id',
                className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter', 'value')])
def update_dashboard(filter):
       if filter == "wr": #water rescue - queries for applicable data
            df = pd.DataFrame(list(shelter.read({'$and': [{"sex_upon_outcome" : "Intact Female"},
                                                {'$or' : [
                                                    {'breed': 'Labrador Retriever Mix'},
                                                    {'breed': 'Chesa Bay Retr Mix'},
                                                    {'breed': 'Newfoundland Mix'}]
                                                },
                                                    {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                             {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                    }]})))
       elif filter == 'mtn': #mountain rescue - queries for applicable data
           df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                               {'$or': [
                                                    {'breed': 'German Shepherd'},
                                                    {'breed': 'Alaskan Malamute'},
                                                    {'breed': 'Old English Sheepdog'},
                                                    {'breed': 'Rottweiler'},
                                                    {'breed': 'Siberian Husky'}]
                                                },
                                                    {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                             {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                }]})))
       elif filter =='dis': #disaster rescue - queries for applicable data
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                {'$or': [
                                                    {'breed': 'Doberman Pinscher'},
                                                    {'breed': 'German Shepherd'},
                                                    {'breed': 'Golden Retriever'},
                                                    {'breed': 'Bloodhound'},
                                                    {'breed': 'Rottweiler'}]
                                                },
                                                    {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                             {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                    }]})))
       elif filter == "RESET" : #Resets data-table to original state when dashboard first launches
            df = pd.DataFrame.from_records(shelter.read({}))
            
       columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
       data=df.to_dict('records')
        
       return (data,columns)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns): #if user clicks on a box, box becomes highlighted
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "data")]
)
def update_graphs(data): #outputs a pie graph of data from data-table
    dff = pd.DataFrame.from_dict(data)
    fig1 = px.pie(dff, names = 'breed',)
    #figure.show()
    return fig1

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, derived_virtual_selected_rows): 
    dff = pd.DataFrame.from_dict(viewData)
    dff = df if viewData is None else pd.DataFrame(viewData)
    animalLoc = None ##initialize list variable
    if not derived_virtual_selected_rows:
        animalLoc = dff.iloc[0]
    else:
        animalLoc = dff.iloc[derived_virtual_selected_rows[0]]
    # variables for tooltip and popup
    lat = animalLoc[13]
    long = animalLoc[14]
    type = animalLoc[3]
    name = animalLoc[9]
    breed = animalLoc[4]
    color = animalLoc[5]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[lat, long], children=[
                dl.Tooltip(type),
                dl.Popup([ #after clicking on Tooltip, pop up with animal info appears
                    html.H1("Animal Info"),
                    html.P(name),
                    html.P(breed),
                    html.P(color),
                ])
            ])
        ])
    ]



app